In [1]:
from spacy.en import English
import matplotlib.pyplot as plot
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import yaml
from fuzzywuzzy import fuzz
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import string

bookname = {"title": "Harry Potter and the Philosopher's Stone", "tag": "book1"}
#bookname = "Harry Potter and the Chamber of Secrets"
parser = English()


In [2]:
# Parse the book
f = open("./book1.txt")
book = dict()
timewords = ["morning", "day", "days", "evening", "night", "week", "weeks", "afternoon"]
scene_number = 0
c = str()
chapter_number = 0
book[scene_number] = dict()
book[scene_number]["title"] = bookname["title"]
for l in f:
    l =str(l)
    if "CHAPTER" in l:
        # strip out chapter headings
        pass
    elif l.isupper() and ("." not in l) and ("?" not in l) and ("!" not in l):
        # new chapter
        print(l)
        scene_number += 1
        book[scene_number] = dict()
        book[scene_number]["title"] = l
        c = str()
    else:
        isTimeskip = False
        for tw in timewords:
            if (("next " + tw) in l) or (("Next " + tw) in l):
                print(l)
                scene_number += 1
                book[scene_number] = dict()
                book[scene_number]["title"] = l
                c = str()
        c += l
        book[scene_number]["text"] = c
book[scene_number] = {"title": l, "text":c}
print(book.keys())

THE BOY WHO LIVED 

"Well, Ted," said the weatherman, "I don't know about that, but it's not only the owls that have been acting oddly today. Viewers as far apart as Kent, Yorkshire, and Dundee have been phoning in to tell me that instead of the rain I promised yesterday, they've had a downpour of shooting stars! Perhaps people have been celebrating Bonfire Night early -- it's not until next week, folks! But I can promise a wet night tonight." 

THE VANISHING GLASS 

Once, Aunt Petunia, tired of Harry coming back from the barbers looking as though he hadn't been at all, had taken a pair of kitchen scissors and cut his hair so short he was almost bald except for his bangs, which she left "to hide that horrible scar." Dudley had laughed himself silly at Harry, who spent a sleepless night imagining school the next day, where he was already laughed at for his baggy clothes and taped glasses. 

Next morning, however, he had gotten up to find his hair exactly as it had been before Aunt Petun


THROUGH THE TRAPDOOR 

THE MAN WITH TWO FACES

MANUAL SCENE BREAK: HARRY WAKES UP

THE END 

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45])


In [3]:
for scene_num in range(1,len(book)):
    print(book[scene_num]["title"])
    print(len(book[scene_num]["text"]))

THE BOY WHO LIVED 

8628
"Well, Ted," said the weatherman, "I don't know about that, but it's not only the owls that have been acting oddly today. Viewers as far apart as Kent, Yorkshire, and Dundee have been phoning in to tell me that instead of the rain I promised yesterday, they've had a downpour of shooting stars! Perhaps people have been celebrating Bonfire Night early -- it's not until next week, folks! But I can promise a wet night tonight." 

17415
THE VANISHING GLASS 

9297
Once, Aunt Petunia, tired of Harry coming back from the barbers looking as though he hadn't been at all, had taken a pair of kitchen scissors and cut his hair so short he was almost bald except for his bangs, which she left "to hide that horrible scar." Dudley had laughed himself silly at Harry, who spent a sleepless night imagining school the next day, where he was already laughed at for his baggy clothes and taped glasses. 

431
Next morning, however, he had gotten up to find his hair exactly as it had be


26154
THROUGH THE TRAPDOOR 

36742
THE MAN WITH TWO FACES

10802
MANUAL SCENE BREAK: HARRY WAKES UP

20508
THE END 

0


In [4]:
from collections import Counter
def people_info(doc):
    people = Counter()
    for ent in doc.ents:
        if ent.label_ in ('CARDINAL', 'ORDINAL'):
            continue
        elif ent.label_ == 'PERSON':
            people[ent.lemma_] += 1
    return people

    

In [15]:
import json
import spacy
import requests
import textacy
from spacy.symbols import nsubj, VERB, dobj
from pprint import pprint
nlp = spacy.load('en')
list_of_people = Counter()
for scene_num in range(1,len(book)):
    print(book[scene_num]["title"])
    doc = nlp(book[scene_num]["text"])
    #print(doc[0])
    #print(doc)
    #print(doc[0].vector.shape)
    #print(doc[0].similarity(doc[2]))
    list_of_people += people_info(doc)
    print(people_info(doc).most_common(3))
    print()
print(list_of_people)

THE BOY WHO LIVED 

[('dursley', 30), ('dudley', 4), ('dursleys', 3)]

"Well, Ted," said the weatherman, "I don't know about that, but it's not only the owls that have been acting oddly today. Viewers as far apart as Kent, Yorkshire, and Dundee have been phoning in to tell me that instead of the rain I promised yesterday, they've had a downpour of shooting stars! Perhaps people have been celebrating Bonfire Night early -- it's not until next week, folks! But I can promise a wet night tonight." 

[('dumbledore', 26), ('mcgonagall', 26), ('dursley', 17)]

THE VANISHING GLASS 

[('harry', 35), ('dudley', 23), ('aunt petunia', 9)]

Once, Aunt Petunia, tired of Harry coming back from the barbers looking as though he hadn't been at all, had taken a pair of kitchen scissors and cut his hair so short he was almost bald except for his bangs, which she left "to hide that horrible scar." Dudley had laughed himself silly at Harry, who spent a sleepless night imagining school the next day, where he


[('harry', 8), ('ron', 7), ('norbert', 6)]

How are you? Thanks for the letter -- I'd be glad to take the Norwegian Ridgeback, but it won't be easy getting him here. I think the best thing will be to send him over with some friends of mine who are coming to visit me next week. Trouble is, they mustn't be seen carrying an illegal dragon. 

[('norbert', 2), ('ridgeback', 1), ('charlie', 1)]

There was a hitch. By the next morning, Ron's bitten hand had swollen to twice its usual size. He didn't know whether it was safe to go to Madam Pomfrey -- would she recognize a dragon bite? By the afternoon, though, he had no choice. The cut had turned a nasty shade of green. It looked as if Norbert's fangs were poisonous. 

[('norbert', 13), ('harry', 12), ('hermione', 10)]

THE FORIBIDDEN FOREST 

[('mcgonagall', 7), ('harry', 7), ('neville', 4)]

A hundred and fifty points lost. That put Gryffindor in last place. In one night, they'd ruined any chance Gryffindor had had for the house cup. Harry 

In [6]:
import nltk
from nltk import sentiment
from collections import defaultdict
import nltk.data



In [14]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
SA = nltk.sentiment.vader.SentimentIntensityAnalyzer()

for scene_num in range(1,len(book)):
    print(book[scene_num]["title"]) #TODO: This isn't helpful, print relevant keywords instead?
    #a = 13
    #b = 10
    #enterkey = ""+chr(a)+chr(b) #VERY HACKY METHOD
    #print(enterkey)
    sentences = tokenizer.tokenize(book[scene_num]["text"])
    #sentences = book[scene_num]["text"].split(enterkey)
    #print(sentences)
    n_counter = 0
    p_counter = 0
    numSentences = 0
    numCharged = 0
    for sentence in sentences: 
        #print("Hello world")
        d = SA.polarity_scores(sentence)
        #print(d)
        numSentences += 1
        if(d == None):
            #print("Nne")
            pass
        elif(d['neu']<0.92 and (d['neu']<0.8 or abs(d['neg']-d['pos']) > 0.05)):
            #print(sentence)
            #print(d)
            numCharged += 1
            #n_counter += d['neg']
            #p_counter += d['pos']
            if d['neg'] < d['pos']:
                p_counter +=1
            else:
                n_counter += 1
    print()
    if numSentences >= 3 and numCharged != 0:
        print(numCharged/numSentences)
        print(n_counter/numCharged)
        print(p_counter/numCharged)
    print()

THE BOY WHO LIVED 


0.49
0.46938775510204084
0.5306122448979592

"Well, Ted," said the weatherman, "I don't know about that, but it's not only the owls that have been acting oddly today. Viewers as far apart as Kent, Yorkshire, and Dundee have been phoning in to tell me that instead of the rain I promised yesterday, they've had a downpour of shooting stars! Perhaps people have been celebrating Bonfire Night early -- it's not until next week, folks! But I can promise a wet night tonight." 


0.3991769547325103
0.41237113402061853
0.5876288659793815

THE VANISHING GLASS 


0.39669421487603307
0.5208333333333334
0.4791666666666667

Once, Aunt Petunia, tired of Harry coming back from the barbers looking as though he hadn't been at all, had taken a pair of kitchen scissors and cut his hair so short he was almost bald except for his bangs, which she left "to hide that horrible scar." Dudley had laughed himself silly at Harry, who spent a sleepless night imagining school the next day, where 



0.3076923076923077
0.625
0.375

How are you? Thanks for the letter -- I'd be glad to take the Norwegian Ridgeback, but it won't be easy getting him here. I think the best thing will be to send him over with some friends of mine who are coming to visit me next week. Trouble is, they mustn't be seen carrying an illegal dragon. 


0.5
0.3333333333333333
0.6666666666666666

There was a hitch. By the next morning, Ron's bitten hand had swollen to twice its usual size. He didn't know whether it was safe to go to Madam Pomfrey -- would she recognize a dragon bite? By the afternoon, though, he had no choice. The cut had turned a nasty shade of green. It looked as if Norbert's fangs were poisonous. 


0.527027027027027
0.5384615384615384
0.46153846153846156

THE FORIBIDDEN FOREST 


0.5365853658536586
0.6363636363636364
0.36363636363636365

A hundred and fifty points lost. That put Gryffindor in last place. In one night, they'd ruined any chance Gryffindor had had for the house cup. Harry fel

In [19]:
newList = list_of_people.most_common(100)
print(newList)

[('harry', 1274), ('ron', 420), ('hermione', 213), ('dumbledore', 113), ('hagrid', 100), ('snape', 98), ('dudley', 96), ('mcgonagall', 93), ('malfoy', 91), ('neville', 87), ('potter', 59), ('uncle vernon', 52), ('quirrell', 51), ('dursley', 50), ('wood', 49), ('dursleys', 42), ('aunt petunia', 34), ('percy', 34), ('gryffindor', 30), ('filch', 30), ('fred', 27), ('norbert', 25), ('harry potter', 23), ('george', 22), ('stone', 21), ('charlie', 20), ('', 17), ('flitwick', 17), ('fang', 17), ('ollivander', 16), ('quidditch', 15), ('hermione granger', 15), ('slytherin', 14), ('ronan', 14), ('piers', 13), ('norris', 13), ('bane', 13), ('muggles', 12), ('hogwarts', 12), ('"', 11), ('nicolas flamel', 11), ('crabbe', 11), ('madam hooch', 11), ('flamel', 11), ('voldemort', 10), ('gringotts', 10), ('goyle', 10), ('quaffle', 10), ('figg', 8), ('yeh', 8), ('griphook', 8), ('hedwig', 8), ('george weasley', 8), ('weasleys', 8), ('bludgers', 8), ('firenze', 8), ('madam pomfrey', 7), ('lily', 7), ('don

In [26]:
import copy

nCounter = copy.deepcopy(list_of_people)
pCounter = copy.deepcopy(list_of_people)
nCounter.subtract(nCounter)
pCounter.subtract(pCounter)

for scene_num in range(1,len(book)):
    sentences = tokenizer.tokenize(book[scene_num]["text"])
    for sentence in sentences: 
        d = SA.polarity_scores(sentence)
        if(d == None):
            pass
        elif(d['neu']<0.92 and (d['neu']<0.8 or abs(d['neg']-d['pos']) > 0.05)):
            for key in list_of_people.keys():
                if key in sentence.lower():
                    if d['neg'] < d['pos']:
                        pCounter[key] +=1
                    else:
                        nCounter[key] += 1
    print()
print(nCounter)
print(pCounter)

x= copy.deepcopy(pCounter)
x.subtract(nCounter)
print(x)














































Counter({'': 1253, 'y': 953, 'st': 485, '"': 416, 'harry': 273, 'ron': 121, 'fr': 96, 'ted': 91, 'hagrid': 68, 'hermione': 61, 'what': 56, 'head': 55, 'snape': 47, 'don': 46, 'neville': 37, 'dudley': 35, 'malfoy': 35, 'quirrell': 31, 'h.': 30, 'vernon': 30, 'dursley': 28, 'uncle vernon': 26, 'mcgonagall': 22, 'gryffindor': 22, 'yeh': 20, 'dumbledore': 19, 'aunt petunia': 18, 'knew': 17, 'voldemort': 15, 'filch': 15, 'dursleys': 14, 'slytherin': 14, 'lee': 14, 'potter': 13, 'wizard': 13, 'stone': 12, 'nasty': 11, 'tom': 11, 'father': 11, 'mother': 11, 'gran': 11, 'weasley': 11, 'wood': 11, 'forge': 11, 'mom': 10, 'peeve': 10, 'quidditch': 9, 'muggle': 9, 'magic': 9, 'curse': 9, 'norbert': 9, 'oak': 8, 'granger': 8, 'flamel': 8, 'mirror': 8, 'muggles': 7, 'gettin': 7, 'hogwarts': 7, 'gringotts': 7, 'percy': 7, 'doi': 7, 'broken': 7, 'baron': 7, 'fang': 6, 'neck': 6, 'gryffindors': 6, 'fluffy': 6, 'broomstick': 6, 'lily': 5, 'piers': 5, 'sir': 

Counter({'': 1332, 'y': 1066, '"': 532, 'st': 487, 'harry': 285, 'fr': 131, 'ron': 125, 'ted': 105, 'hagrid': 75, 'what': 70, 'hermione': 64, 'dumbledore': 58, 'don': 54, 'head': 37, 'gryffindor': 37, 'h.': 34, 'snape': 34, 'yeh': 32, 'dudley': 28, 'neville': 26, 'potter': 25, 'vernon': 25, 'hogwarts': 25, 'malfoy': 25, 'uncle vernon': 24, 'mcgonagall': 22, 'wizard': 22, 'slytherin': 21, 'lee': 21, 'mom': 21, 'dursley': 20, 'stone': 20, 'weasley': 19, 'oak': 18, 'bright': 18, 'magic': 17, 'wood': 17, 'quirrell': 16, 'quidditch': 16, 'knew': 14, 'muggle': 13, 'gettin': 11, 'fred': 11, 'filch': 10, 'gran': 10, 'forge': 10, 'dursleys': 9, 'crabbe': 9, 'goyle': 9, 'flitwick': 9, 'flamel': 9, 'father': 8, 'hufflepuff': 8, 'george': 8, 'peeve': 8, 'mirror': 8, 'broomstick': 8, 'gringotts': 7, 'tom': 7, 'percy': 7, 'charlie': 7, 'doi': 7, 'norbert': 7, 'madam pomfrey': 6, 'aunt petunia': 6, 'sir': 6, 'ollivander': 6, 'mother': 6, 'seamus': 6, 'norris': 6, 'neck': 6, 'firenze': 6, 'nicolas': 6

Counter({'"': 116, 'y': 113, '': 79, 'dumbledore': 39, 'fr': 35, 'hogwarts': 18, 'bright': 16, 'gryffindor': 15, 'ted': 14, 'what': 14, 'potter': 12, 'harry': 12, 'yeh': 12, 'mom': 11, 'oak': 10, 'wizard': 9, 'don': 8, 'magic': 8, 'fred': 8, 'crabbe': 8, 'weasley': 8, 'stone': 8, 'hagrid': 7, 'quidditch': 7, 'slytherin': 7, 'lee': 7, 'madam pomfrey': 6, 'goyle': 6, 'flitwick': 6, 'wood': 6, 'norris': 5, 'quaffle': 5, 'h.': 4, 'gettin': 4, 'muggle': 4, 'hedwig': 4, 'ron': 4, 'charlie': 4, 'hufflepuff': 3, 'george': 3, 'hermione': 3, 'ravenclaw': 3, 'nick': 3, 'nearly headless': 3, 'seamus': 3, 'headless nick': 3, 'lee jordan': 3, 'bludgers': 3, 'bell': 3, 'firenze': 3, 'nicolas': 3, 'albus dumbledore': 2, 'duddy': 2, 'budge': 2, 'goblin': 2, 'mmm': 2, 'ollivander': 2, 'algie': 2, 'madam hooch': 2, 'slytherins': 2, 'remembrall': 2, 'parvati patil': 2, 'fine': 2, 'lavender': 2, 'jordan': 2, 'madam pince': 2, 'gred': 2, 'perenelle': 2, 'st': 2, 'broomstick': 2, 'rejoice': 1, 'kent': 1, 'al

In [27]:
#very strange... 
#most people are pretty well categorized by this but somehow Crabbe and Goyle are positive and Neville is negative???
#other interesting observations:
#people's first names are used in positive contexts, but when someone calls "Miss Granger" or "Granger" it is often negative

